In [0]:
"""
Minimal character-level LSTM model. Written by Ngoc Quan Pham
Code structure borrowed from the Vanilla RNN model from Andreij Karparthy @karparthy.
BSD License
"""
import numpy as np
from random import uniform
import sys

In [0]:
# Since numpy doesn't have a function for sigmoid
# We implement it manually here
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# The derivative of the sigmoid function
def dsigmoid(y):
    return y * (1 - y)


# The derivative of the tanh function
def dtanh(x):
    return 1 - x*x


# The numerically stable softmax implementation
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [27]:
# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 1115394 characters, 65 unique.


**此处有疑问，sys.argv怎么用？？（sys.argv[1]是我们run: "python lstm_py train"，其中的train,用于设置输入参数，选择不同的模式，比如train or gradientcheck）**

In [0]:
option = sys.argv[1]

In [0]:
# hyperparameters
emb_size = 4
hidden_size = 512  # size of hidden layer of neurons
seq_length = 64  # number of steps to unroll the RNN for
learning_rate = 5e-2
max_updates = 500000
std = 0.1

concat_size = emb_size + hidden_size

In [0]:
# model parameters
# char embedding parameters
Wex = np.random.randn(emb_size, vocab_size)*std # embedding layer

# LSTM parameters
Wf = np.random.randn(hidden_size, concat_size) * std # forget gate
Wi = np.random.randn(hidden_size, concat_size) * std # input gate
Wo = np.random.randn(hidden_size, concat_size) * std # output gate
Wc = np.random.randn(hidden_size, concat_size) * std # c term

bf = np.zeros((hidden_size, 1)) # forget bias
bi = np.zeros((hidden_size, 1)) # input bias
bo = np.zeros((hidden_size, 1)) # output bias
bc = np.zeros((hidden_size, 1)) # memory bias

# Output layer parameters
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
by = np.zeros((vocab_size, 1)) # output bias

## 需要我们补充的部分1：向前传播

In [0]:
def forward(inputs, targets, memory):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """

    # The LSTM is different than the simple RNN that it has two memory cells
    # so here you need two different hidden layers
    hprev, cprev = memory

    # Here you should allocate some variables to store the activations during forward
    # One of them here is to store the hiddens and the cells
    xs, cs, hs, os, ps, ys, wes, zs, fgate, igate, chat, ogate = {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}

    hs[-1] = np.copy(hprev)
    cs[-1] = np.copy(cprev)

    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1

        # convert word indices to word embeddings
        wes[t] = np.dot(Wex, xs[t])

        # LSTM cell operation
        # first concatenate the input and h
        # This step is irregular (to save the amount of matrix multiplication we have to do)
        # I will refer to this vector as [h X]
        zs[t] = np.row_stack((hs[t-1], wes[t]))

        # YOUR IMPLEMENTATION should begin from here

        # compute the forget gate
        fgate[t] = sigmoid(np.dot(Wf, zs[t]) + bf)

        # compute the input gate
        igate[t] = sigmoid(np.dot(Wi, zs[t]) + bi)

        # compute the candidate memory
        chat[t] = np.tanh(np.dot(Wc, zs[t]) + bc)

        # new memory: applying forget gate on the previous memory
        # and then adding the input gate on the candidate memory
        cs[t] = fgate[t] * cs[t-1] + igate[t] * chat[t]

        # output gate
        ogate[t] = sigmoid(np.dot(Wo, zs[t]) + bo)

        # new hidden state for the LSTM
        hs[t] = ogate[t] * np.tanh(cs[t])

        # DONE LSTM
        # output layer - softmax and cross-entropy loss
        # unnormalized log probabilities for next chars

        os[t] = np.dot(Why, hs[t]) + by

        # softmax for probabilities for next chars
        ps[t] = softmax(os[t])

        # cross-entropy loss
        # cross entropy loss at time t:
        ys[t] = np.zeros((vocab_size, 1))
        
        # create an one hot vector for the label y
        ys[t][targets[t]] = 1

        # and then cross-entropy (see the elman-rnn file for the hint)
        loss_t = np.sum(-np.log(ps[t]) * ys[t])
        loss += loss_t

    # define your activations
    memory = (hs[len(inputs)-1], cs[len(inputs)-1])
    activations = (xs, cs, hs, os, ps, ys, wes, zs, fgate, igate, chat, ogate)

    return loss, activations, memory

## 需要我们补充的部分2：向后传播

In [0]:
def backward(activations, clipping=True):

    # backward pass: compute gradients going backwards
    # Here we allocate memory for the gradients
    dWex, dWhy = np.zeros_like(Wex), np.zeros_like(Why)
    dby = np.zeros_like(by)
    dWf, dWi, dWc, dWo = np.zeros_like(Wf), np.zeros_like(Wi),np.zeros_like(Wc), np.zeros_like(Wo)
    dbf, dbi, dbc, dbo = np.zeros_like(bf), np.zeros_like(bi),np.zeros_like(bc), np.zeros_like(bo)

    xs, cs, hs, os, ps, ys, wes, zs, fgate, igate, chat, ogate = activations
    
    # similar to the hidden states in the vanilla RNN
    # We need to initialize the gradients for these variables
    dh = np.zeros_like(hs[0])
    dc = np.zeros_like(cs[0])
    dz = np.zeros_like(zs[0])
    
    # back propagation through time starts here
    for t in reversed(range(len(inputs))):

        # IMP2LEMENT YOUR BACKPROP HERE
        # refer to the file elman_rnn.py for more details
        
        # 1. equ.
        do = ps[t] - ys[t]
        # 2. equ.
        dWhy += np.dot(do, hs[t].T)
        dby += do
        dh += np.dot(Why.T, do)
        # 3. equ.
        dogate = dh * np.tanh(cs[t]) # false
        dc += dh * ogate[t] * dtanh(np.tanh(cs[t])) 
        # 4. equ.
        dWo += np.dot(dogate * dsigmoid(ogate[t]), zs[t].T) # false
        dbo += dogate * dsigmoid(ogate[t])
        dz += np.dot(Wo.T, dogate * dsigmoid(ogate[t]))
        # 5. equ.
        dfgate = dc * cs[t-1]
        digate = dc * chat[t]
        dchat = dc * igate[t]
        dc = dc * fgate[t]
        # 6. equ.
        dWc += np.dot(dchat * dtanh(chat[t]), zs[t].T)
        dz += np.dot(Wc.T, dchat * dtanh(chat[t]))
        dbc += dchat * dtanh(chat[t])
        # 7. equ.
        dWi += np.dot(digate * dsigmoid(igate[t]), zs[t].T)
        dz += np.dot(Wi.T, digate * dsigmoid(igate[t]))
        dbi += digate * dsigmoid(igate[t])
        # 8. equ.
        dWf += np.dot(dfgate * dsigmoid(fgate[t]), zs[t].T)
        dz += np.dot(Wf.T, dfgate * dsigmoid(fgate[t]))
        dbf += dfgate * dsigmoid(fgate[t])
        # 9. equ.
        dWex += np.dot(dz[len(hs[t-1]):], xs[t].T)
        dh = dz[:len(hs[t-1])]
        dz = 0
        
        if clipping:
            # clip to mitigate exploding gradients
            for dparam in [dWex, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWhy, dby]:
                np.clip(dparam, -5, 5, out=dparam)

    gradients = (dWex, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWhy, dby)
    
#     if test:
#         # Test of 2. equ.
#         print('Test of 2. equ.')
        

    return gradients

## 需要我们补充的部分3：sample

In [0]:
def sample(memory, seed_ix, n):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    """
    h, c = memory
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    
    generated_chars = []

    for t in range(n):
        # IMPLEMENT THE FORWARD FUNCTION ONE MORE TIME HERE
        # BUT YOU DON"T NEED TO STORE THE ACTIVATIONS
        wes = np.dot(Wex, x)
        zs = np.row_stack((h, wes))
        fgate = sigmoid(np.dot(Wf, zs) + bf)
        igate = sigmoid(np.dot(Wi, zs) + bi)
        chat = np.tanh(np.dot(Wc, zs) + bc)
        c = fgate * c + igate * chat
        ogate = sigmoid(np.dot(Wo, zs) + bo)
        h = ogate * np.tanh(c)
        
        o = np.dot(Why, h) + by
        p = softmax(o)

        # the the distribution, we randomly generate samples:
        ix = np.random.multinomial(1, p.ravel())
        x = np.zeros((vocab_size, 1))

        for j in range(len(ix)):
            if ix[j] == 1:
                index = j
        x[index] = 1
        generated_chars.append(index)

    return generated_chars

In [0]:
option = 'train'

if option == 'train':
    n, p = 0, 0
    n_updates = 0

    # momentum variables for Adagrad
    mWex, mWhy = np.zeros_like(Wex), np.zeros_like(Why)
    mby = np.zeros_like(by) 

    mWf, mWi, mWo, mWc = np.zeros_like(Wf), np.zeros_like(Wi), np.zeros_like(Wo), np.zeros_like(Wc)
    mbf, mbi, mbo, mbc = np.zeros_like(bf), np.zeros_like(bi), np.zeros_like(bo), np.zeros_like(bc)

    smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
    
    min_loss = 0

    while True:
        # prepare inputs (we're sweeping from left to right in steps seq_length long)
        if p+seq_length+1 >= len(data) or n == 0:
            hprev = np.zeros((hidden_size,1)) # reset RNN memory
            cprev = np.zeros((hidden_size,1))
            p = 0 # go from start of data
        inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
        targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

        # sample from the model now and then
        if n % 100 == 0:
            sample_ix = sample((hprev, cprev), inputs[0], 200)
            txt = ''.join(ix_to_char[ix] for ix in sample_ix)
            print ('----\n %s \n----' % (txt, ))

        # forward seq_length characters through the net and fetch gradient
        loss, activations, memory = forward(inputs, targets, (hprev, cprev))
        gradients = backward(activations, clipping=True)

        hprev, cprev = memory
        dWex, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWhy, dby = gradients
        smooth_loss = smooth_loss * 0.999 + loss * 0.001
        
        if smooth_loss < min_loss:
            min_loss = smooth_loss
            min_loss_iter = n
        if n % 100 == 0: 
            print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
            print('minimum loss: %f at iteration %i' %(min_loss, min_loss_iter))

        # perform parameter update with Adagrad
        for param, dparam, mem in zip([Wf, Wi, Wo, Wc, bf, bi, bo, bc, Wex, Why, by],
                                    [dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWex, dWhy, dby],
                                    [mWf, mWi, mWo, mWc, mbf, mbi, mbo, mbc, mWex, mWhy, mby]):
            mem += dparam * dparam
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

        p += seq_length # move data pointer
        n += 1 # iteration counter
        n_updates += 1
        if n_updates >= max_updates:
            break

elif option == 'gradcheck':

    p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    delta = 0.001

    hprev = np.zeros((hidden_size, 1))
    cprev = np.zeros((hidden_size, 1))

    memory = (hprev, cprev)

    loss, activations, _ = forward(inputs, targets, memory)
    gradients = backward(activations, clipping=False)
    dWex, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWhy, dby = gradients

    for weight, grad, name in zip([Wf, Wi, Wo, Wc, bf, bi, bo, bc, Wex, Why, by], 
                                   [dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWex, dWhy, dby],
                                   ['Wf', 'Wi', 'Wo', 'Wc', 'bf', 'bi', 'bo', 'bc', 'Wex', 'Why', 'by']):

        str_ = ("Dimensions dont match between weight and gradient %s and %s." % (weight.shape, grad.shape))
        assert(weight.shape == grad.shape), str_

        wrong_flag = False
        
        print(name)
        for i in range(weight.size):
            # evaluate cost at [x + delta] and [x - delta]
            w = weight.flat[i]
            weight.flat[i] = w + delta
            loss_positive, _, _ = forward(inputs, targets, memory)
            weight.flat[i] = w - delta
            loss_negative, _, _ = forward(inputs, targets, memory)
            weight.flat[i] = w  # reset old value for this parameter

            grad_analytic = grad.flat[i]
            grad_numerical = (loss_positive - loss_negative) / ( 2 * delta )

            # compare the relative error between analytical and numerical gradients
            rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)

            if rel_error > 0.01:
#                 if name == 'Wo' or name == 'bo':
                print ('WARNING %f, %f => %e ' % (grad_numerical, grad_analytic, rel_error))
#                     print('i=%i' %i)
#                     print('numerical: %f, analytic: %f' %(grad_numerical * 1000, grad_analytic * 1000))
                wrong_flag = True
    
        if wrong_flag:
            print('This gradient is not correct!')
        else:
            print('Test pass!')
            

#     weight, grad, name = Wo, dWo, 'Wo'
    
#     str_ = ("Dimensions dont match between weight and gradient %s and %s." % (weight.shape, grad.shape))
#     assert(weight.shape == grad.shape), str_

#     wrong_flag = False

#     print(name)
#     for i in range(weight.size):
#         # evaluate cost at [x + delta] and [x - delta]
#         w = weight.flat[i]
#         weight.flat[i] = w + delta
#         loss_positive, _, _ = forward(inputs, targets, memory)
#         weight.flat[i] = w - delta
#         loss_negative, _, _ = forward(inputs, targets, memory)
#         weight.flat[i] = w  # reset old value for this parameter

#         grad_analytic = grad.flat[i]
#         grad_numerical = (loss_positive - loss_negative) / ( 2 * delta )

#         # compare the relative error between analytical and numerical gradients
#         rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)

#         if rel_error > 0.01:
# #                     print ('WARNING %f, %f => %e ' % (grad_numerical, grad_analytic, rel_error))
#             print('i=%i' %i)
#             print('numerical: %f, analytic: %f, rel_error: %f' %(grad_numerical * 1000, grad_analytic * 1000, rel_error))
#             wrong_flag = True
            
#     if wrong_flag:
#         print("This gradient is not correct!")
#     else:
#         print("Test pass!")

----
 nfGdk:MB;BQ!Cb!cUevmI'DEoNB?vc
!&Dhl;FcT?HK.!'KVT c JmDdy'?hPS-JnF?EDhw:Z hNyYdI3TLqYFRMSdljFR!AQoTmd:icW!S!LHl.dziYyHToXTQpXM'd-:k?SV'esjsbTVf':Sif'hwN$&BFIILMyXFaY!WrLOfyFWPoZcfr,MTawdic;UJZpSWQgHOg 
----
iter 0, loss: 267.160774
----
 neal heN pl he s
a  ud ld, ch dunyesrtrg tz IuiAph
l  
aev cel to'eedtaliu
h z ntde?  f
setbu
N
bn ewtBtCuhen yel ne  aiale 
ueap-ft,rla
 aucoi,el ,
 ttl sona,  th t
cr tBie :ygTt neohlktiC ld  mr 
ed 
----
iter 100, loss: 263.108793
----
 eeluh wty a g Ins
ihesladee feitkes h wyah,ysstm thl hotl wos ioh nond  oui
HoaniS tonl:rn
Bos
Mhte.i sos nh soaharou-nrr hoCm toei nud 
hotls hipwhd rnam H d mlil nierhd thlmas sicd tao ne aiIoan ooh 
----
iter 200, loss: 255.872668
----
 sw 
VhiRhe wwsscnd gnde feah tane binrl yer heib yu
IAwis,ree heu oel  teutauhatat miw sou hor binit atatitt as toucglde  reld hot, hin Teveh y uhe wir hetaw sawls.sirote
sev ghey hal iheraityirt bit  
----
iter 300, loss: 248.055949
----
 t iw cougth mle gount.
tolish fro fh

In [0]:
# import os
# import pprint
# import tensorflow as tf

# if 'COLAB_TPU_ADDR' not in os.environ:
#   print('ERROR: Not connected to a TPU runtime')
# else:
#   tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
#   print ('TPU address is', tpu_address)

#   with tf.Session(tpu_address) as session:
#     devices = session.list_devices()

#   print('TPU devices:')
#   pprint.pprint(devices)